In [1]:
import mysql.connector
import pandas as pd
import os
import dotenv
from dotenv import dotenv_values
from io import StringIO
import boto3

dotenv.load_dotenv()
config = dotenv_values()

In [2]:
params = {
    'host': config['HOST'],
    'database': config['DATABASE'],
    'user': config['USER'],
    'password': config['PASSWORD']
}

conn = mysql.connector.connect(**params)
cur = conn.cursor()

In [3]:
aws_id = config['AWS_ID']
aws_secret = config['AWS_SECRET']

client = boto3.client('s3',
                      aws_access_key_id=aws_id,
                      aws_secret_access_key=aws_secret)

In [4]:
create_table_dict = {
    'prod_store': 
    """
    CREATE TABLE prod_store (
        id VARCHAR(32) NOT NULL,
        item_nbr INT NOT NULL,
        DESCRICAO VARCHAR(255) NOT NULL,
        SECAO VARCHAR(255) NOT NULL,
        GRUPO VARCHAR(255) NOT NULL,
        SUBGRUPO VARCHAR(255) NOT NULL,
        store_nbr INT NOT NULL,
        avg_unit_sales DECIMAL(10 , 2 ) NULL,
        avg_price DECIMAL(10 , 2 ) NULL,
        PRIMARY KEY (id)
    );
    """,
    'experiments': 
    """
    CREATE TABLE experiments (
        id VARCHAR(255) NOT NULL,
        item_nbr INT NOT NULL,
        store_nbr INT NOT NULL,
        problem_complexity VARCHAR(32) NOT NULL,
        model_performance VARCHAR(32) NOT NULL,
        input_complexity VARCHAR(32) NOT NULL,
        PRIMARY KEY (id)
    );
    """,
    'experiments_context':
    """
    CREATE TABLE experiments_context (
        id INT NOT NULL,
        experiment_id VARCHAR(255) NOT NULL,
        date DATE NOT NULL,
        unit_sales DECIMAL(10 , 2 ) NULL,
        unit_sales_pred DECIMAL(10 , 2 ) NULL,
        price DECIMAL(10 , 2 ) NULL,
        PRIMARY KEY (id)
    );
    """,
    'experiments_history':
    """
    CREATE TABLE experiments_history (
        id INT NOT NULL,
        experiment_id VARCHAR(255) NOT NULL,
        date DATE NOT NULL,
        unit_sales DECIMAL(10 , 2 ) NULL,
        price DECIMAL(10 , 2 ) NULL,
        years_ago INT NOT NULL,
        PRIMARY KEY (id)
    );
    """,
    'experiments_history_ts':
    """
    CREATE TABLE experiments_history_ts (
        id INT NOT NULL,
        experiment_id VARCHAR(255) NOT NULL,
        date DATE NOT NULL,
        unit_sales DECIMAL(10 , 2 ) NULL,
        PRIMARY KEY (id)
    );
    """,
    'avg_dow_sales': 
    """
    CREATE TABLE avg_dow_sales (
        id INT NOT NULL,
        item_nbr INT NOT NULL,
        store_nbr INT NOT NULL,
        dow VARCHAR(32) NOT NULL,
        avg_dow_sales DECIMAL(10 , 2 ) NULL,
        PRIMARY KEY (id)
    );
    """,
    'avg_wom_sales':
    """
    CREATE TABLE avg_wom_sales (
        id INT NOT NULL,
        item_nbr INT NOT NULL,
        store_nbr INT NOT NULL,
        wom INT NOT NULL,
        avg_wom_sales DECIMAL(10 , 2 ) NULL,
        PRIMARY KEY (id)
    );
    """,
    'avg_monthly_sales':
    """
    CREATE TABLE avg_monthly_sales (
        id INT NOT NULL,
        item_nbr INT NOT NULL,
        store_nbr INT NOT NULL,
        month INT NOT NULL,
        avg_monthly_sales DECIMAL(10 , 2 ) NULL,
        PRIMARY KEY (id)
    );
    """
}

In [5]:
def read_data_from_aws_bucket(client, file_name):
    bucket_name = 'decisionhelper'
    object_key = file_name
    
    csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')

    df = pd.read_csv(StringIO(csv_string))
    
    return df

In [6]:
def get_cols_inserted(n_cols):
    cols_inserted = ''
    for i in range(n_cols):
        if i == 0:
            cols_inserted += '(%s, '
        elif 0 < i < n_cols-1:  
            cols_inserted += '%s, '
        else:
            cols_inserted += '%s)'
            
    return cols_inserted

In [7]:
for table, operation in create_table_dict.items():
    try:
        cur.execute(operation)
        print(f'Table {table} created successfully')
        
        file_name = table + '.csv'
        df = read_data_from_aws_bucket(client, file_name)
        
        n_cols = len(list(df.columns))
        cols_inserted = get_cols_inserted(n_cols)
        
        operation = f'INSERT INTO {table} VALUES {cols_inserted}'
        
        rows = df.values.tolist()
        for row in rows:
            cur.execute(
                operation, row
            )

        conn.commit()
        
        print(f'{len(rows)} rows created \n')
    except Exception as e:
        print(e)

Table prod_store created successfully
1664 rows created 

Table experiments created successfully
1210 rows created 

Table experiments_context created successfully
8470 rows created 

Table experiments_history created successfully
16940 rows created 

Table experiments_history_ts created successfully
272720 rows created 

Table avg_dow_sales created successfully
11648 rows created 

Table avg_wom_sales created successfully
8320 rows created 

Table avg_monthly_sales created successfully
19968 rows created 

